In [3]:
import gym
import numpy as np
import time
from IPython import display
import time

In [4]:
env1 = gym.make('FrozenLake-v1')

In [5]:
def play(env, policy, render=False):
    state = env.reset()
    total_reward = 0
    steps = 0
    done = False
    while not done:
        action = policy[state]
        next_state, reward, done, info = env.step(action)
        total_reward += reward
        steps += 1
        if render:
            env.render()
            time.sleep(0.2)
            if not done:
                display.clear_output(wait=True)
        state = next_state

    return (total_reward, steps)

In [6]:
def play_multiple_times(env, policy, max_episodes):
    success = 0
    list_of_steps = []
    for i in range(max_episodes):
        total_reward, steps = play(env, policy)

        if total_reward > 0:
            success += 1
            list_of_steps.append(steps)

    print(f'Number of successes: {success}/{max_episodes}')
    print(f'Average number of steps: {np.mean(list_of_steps)}')

In [7]:
def policy_evaluation(env, policy, max_iters=500, gamma=0.9):
    # Initialize the values of all states to be 0
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # Update the value of each state
        for state in range(env.observation_space.n):
            action = policy[state]

            # Compute the q-value of the action
            q_value = 0
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * prev_v_values[next_state])

            v_values[state] = q_value # update v-value
        
        # Check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break
    
    return v_values

In [8]:
def value_iteration(env, max_iters=500, gamma=0.9):
    # initialize
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters): 
        prev_v_values = np.copy(v_values)

        # update the v-value for each state
        for state in range(env.observation_space.n):
            q_values = []
            
            # compute the q-value for each action that we can perform at the state
            for action in range(env.action_space.n):
                q_value = 0
                # loop through each possible outcome
                for prob, next_state, reward, done in env.P[state][action]:
                    q_value += prob * (reward + gamma * prev_v_values[next_state])
                
                q_values.append(q_value)
            
            # select the max q-values
            best_action = np.argmax(q_values)
            v_values[state] = q_values[best_action]
        
        # check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break
    
    return v_values

In [9]:
def policy_extraction(env, v_values, gamma=0.9):
    # initialize
    policy = np.zeros(env.observation_space.n)

    # loop through each state in the environment
    for state in range(env.observation_space.n):
        q_values = []
        # loop through each action
        for action in range(env.action_space.n):
            q_value = 0
            # loop each possible outcome
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * v_values[next_state])
            
            q_values.append(q_value)
        
        # select the best action
        best_action = np.argmax(q_values)
        policy[state] = best_action
    
    return policy

In [13]:
start=time.time()
optimal_v_values1=value_iteration(env1)
end=time.time()
print('Time: ',end-start + 1)
optimal_policy1=policy_extraction(env1, optimal_v_values1, gamma=0.9)
play_multiple_times(env1, optimal_policy1, 1000)

Converged at 79-th iteration.
Time:  1.0349922180175781
Number of successes: 741/1000
Average number of steps: 37.465587044534416


In [11]:
def policy_iteration(env, max_iters=500, gamma=0.9):
    # Khoi tao v_values va bang policy
    v_values = np.zeros(env.observation_space.n)
    policy = np.zeros(env.observation_space.n)
    
    # Iterate cho den khi nao la du nhieu thi dung
    for i in range(max_iters): 
        #Tao ban copy
        prev_policy=np.copy(policy)
        # Converge Vpi
        for j in range(max_iters):
            prev_v_values = np.copy(v_values)
            for state in range(env.observation_space.n):

                # Tuan theo chien luoc pi
                action = policy[state]
                v_value = 0

                # Loop moi action trong state s
                for prob, next_state, reward, done in env.P[state][action]:
                    v_value += prob * (reward + gamma * prev_v_values[next_state])

                v_values[state]=v_value
            # Kiem tra converge Vpi
            if np.all(np.isclose(v_values, prev_v_values)):
                break
        
        #Improvement chien luoc pi
        for state in range(env.observation_space.n):
            values=[]
            for action in range(env.action_space.n):
                v_value = 0
                # Loop moi action trong state s
                for prob, next_state, reward, done in env.P[state][action]:
                    v_value += prob * (reward + gamma * v_values[next_state])
                values.append(v_value)
            policy[state]=np.argmax(values)
        # Kiem tra policy converge    
        if (prev_policy==policy).all():
            break
    
    return policy

In [12]:
start=time.time()
policy_iter1=policy_iteration(env1)
end=time.time()
print('Time: ',end-start + 1)
play_multiple_times(env1, policy_iter1, 1000)

Time:  1.0280559062957764
Number of successes: 750/1000
Average number of steps: 37.32


In [15]:
env2 = gym.make('FrozenLake8x8-v1')

In [16]:
start=time.time()
optimal_v_values2=value_iteration(env2)
end=time.time()
print('Time: ',end-start + 1)
optimal_policy2=policy_extraction(env2, optimal_v_values2, gamma=0.9)
play_multiple_times(env2, optimal_policy2, 1000)

Converged at 117-th iteration.
Time:  1.190941572189331
Number of successes: 744/1000
Average number of steps: 72.78494623655914


In [17]:
start=time.time()
policy_iter2=policy_iteration(env2)
end=time.time()
print('Time: ',end-start + 1)
play_multiple_times(env2, policy_iter2, 1000)

Time:  1.1691973209381104
Number of successes: 723/1000
Average number of steps: 70.65421853388658


In [19]:
env3 = gym.make('Taxi-v3')

In [20]:
start=time.time()
optimal_v_values3=value_iteration(env3)
end=time.time()
print('Time: ',end-start + 1)
optimal_policy3=policy_extraction(env3, optimal_v_values3, gamma=0.9)
play_multiple_times(env3, optimal_policy3, 1000)

Converged at 116-th iteration.
Time:  2.4231574535369873
Number of successes: 1000/1000
Average number of steps: 13.011


In [22]:
start=time.time()
policy_iter3=policy_iteration(env3)
end=time.time()
print('Time: ',end-start + 1)
play_multiple_times(env3, policy_iter3, 1000)

Time:  1.6230072975158691
Number of successes: 1000/1000
Average number of steps: 13.028


Nhận xét:

Độ chính xác của Value Iteration và Policy Iteration là như nhau vì nó cũng sẽ converge về optimal solution, tùy thuộc vào số iteration

Tuy nhiên, Policy Iteration sẽ chạy nhanh hơn Value Iteration nhưng có số vòng lặp nhiều hơn vì sẽ Policy sẽ phải converge về giá trị kì vọng nhận được khi tuân theo chiến lược Pi và sau đó sẽ phải improve chiến lược Pi để chiến lược hội tụ về Pi*
